**Table of contents**

* [Attention](#1)
    * [Definition](#1_1)
    * [Motivation](#1_2)
    * [General idea behind attention](#1_3)
    * [Dot product as a measure of similarity](#1_4)
* [Implementations](#2)
    * [Bhadanau's](#2_1) [Bhadanau, et al (2014)](https://arxiv.org/abs/1409.0473)
        * [Alignment scores](#2_1_1)
        * [Create the inputs](#2_1_2)
        * [First Layer: activations](#2_1_3)
        * [Second Layer: scores](#2_1_4)
        * [Calculate weights: Softmax function](#2_1_5)
        * [Apply the weights to the encoder's hidden states](#2_1_6)
        * [Code](#2_1_7)
    * [Scaled dot-product attention](#2_2)
        * [Code](#2_2_1)
        * [Visualizing how the attention relates similar words from inputs and outputs](#2_2_2)    
            * [Aligned vectors](#2_2_2_1)
            * [Calculating embedding matrices for our concrete inputs and outputs](#2_2_2_2)
            * [Code](#2_2_2_3)

# Attention <a class="anchor" id="1"></a>

## Definition <a class="anchor" id="1_1"></a>
Attention is a mechanism that let's the model focus on specific parts of the input sequence when generating the next output. More concretely, since we are training a model, attention will help the model focus on the most relevant parts (according to the problem at hand) of the input when generating the next outpput.

Attention is not exlcusive to _seq2seq_ models, neither is it to _NLP_. The rest of the document will focus on _NLP_ applications though.

## Motivation <a class="anchor" id="1_2"></a>
The attention mechanism was designed to address the information bottleneck problem suffered by _seq2seq_ models , particularly, with long input sequences.

In the original _seq2seq_ networks with an encoder-decoder architecture, the encoder would only pass the last step's hidden state vector to the decoder. This means that a single hidden vector was used to encode every other hidden vector from previous steps. This resulted in low performarce as the size of the input sequences increased.

<img src="./images/information-bottleneck.PNG" alt="Information bottleneck" width="80%" />

## General idea behind attention<a class="anchor" id="1_3"></a>
Attention proposes to use all of the encoder's hidden state vectors together with the decoder's hidden state from the previous output step to form a weighted sum that gives more importance (weight) to the most relevant encoder hidden state vectors.
    
<img src="./images/attention-hidden-states.PNG" alt="Attention combines encoder and decoder states into a context" width="80%" />    


At least, the two attention mechanisms discussed in this article follow the next ideas:
- Calculate a measure of similarity between inputs and outputs of the model, called Aligment.
- Calculate a set of scores for each of those alignments. Scores are values between 0 and 1.
- Use the scores as weights to weight the inputs so that the most relevant ones have higher values.
- Sum the weighted inputs to form a context vector that gets passed to the decoder.

<img src="./images/attention-general-steps.PNG" alt="Attention's general steps" width="80%" />

## Dot product as a measure of similarity<a class="anchor" id="1_4"></a>
Both types of attention mechanisms discussed here use the dot product of matrices as a measure of similarity.

$$
\large V·W = \lVert V \rVert·\lVert W \rVert \cos(\alpha)
$$

By definition, the dot product of orthogonal vectors is zero. And the more they go in the same direction the larger it is. The sign indicate same or opposite directions.



# Implementations<a class="anchor" id="2"></a>

## Bhadanau's Attention:<a class="anchor" id="2_1"></a>
The goal of the attention layer is to produce a context vector that contains the relevant information from the encoder's state. It consists of the following stages:
- Calculate the alignments using a feedforward neural network
- Apply a softmax function to the alignments to calculate some weights
- Use the previous weights to scale the encoder's hidden vectors

In this approach, the alignments are learnt using a neural network. Therefore, some weights are learnt to maximize the similarity measure between the encoder's hidden state vectors and the decoder's previous state hidden vector.

### Alignment scores<a class="anchor" id="2_1_1"></a>
This is a measure of similarity between the decoder hidden state and each encoder hidden state. The operation is designed in the the paper like below.

$$
\large e_{ij} = v_a^\top \tanh{\left(W_a s_{i-1} + U_a h_j\right)}
$$

- $W_a \in \mathbb{R}^{n\times m}$, $U_a \in \mathbb{R}^{n \times m}$, and $v_a \in \mathbb{R}^m$
are the weight matrices. 
- $n$ is the hidden state size. 

e_ij is a score of how well the inputs around j match the expected output at i. It is learnable. In practice, this is implemented as a feedforward neural network with two layers, where $m$ is the size of the layers in the alignment network.
- Create the inputs, concatenating the encoder hidden vectors with the decoder's previous step hidden vector repeated.
- Calculate the first layer of the FNN: the activations
- Calculate the second layer: the scores.

### Create the inputs<a class="anchor" id="2_1_2"></a>
s_i-1 is repearted along axis 0 (rows) to form a matrix that can be concatenated with the hidden states along axis 1 (columns).

$$
\mathbf{Inputs}
=
\begin{bmatrix}
\cdots & h_1    & \cdots \\
\cdots & h_2    & \cdots \\
\cdots & h_3    & \cdots \\
\vdots & \vdots & \vdots \\
\cdots & h_n    & \cdots \\
\end{bmatrix}
+
\begin{bmatrix}
\cdots & s_{i-1} & \cdots \\
\cdots & s_{i-1} & \cdots \\
\cdots & s_{i-1} & \cdots \\
\vdots & \vdots  & \vdots \\
\cdots & s_{i-1} & \cdots \\
\end{bmatrix}
$$

- The encoder's hidden state's shape is (n, m)
- The decoder's hidden state's shape is (n, m)
- Inputs' shape is (n, 2*m). We are concatenating, not multiplicating.

### First layer: activations <a class="anchor" id="2_1_3"></a>
$$
\mathbf{scores}
=
\begin{bmatrix}
\cdots & \cdots    & \cdots \\
\cdots & \cdots    & \cdots \\
\cdots & Inputs    & \cdots \\
\vdots & \vdots    & \vdots \\
\cdots & \cdots    & \cdots \\
\end{bmatrix}
·
\begin{bmatrix}
\cdots & \cdots    & \cdots \\
\cdots & \cdots    & \cdots \\
\cdots & Layer 1    & \cdots \\
\vdots & \vdots    & \vdots \\
\cdots & \cdots    & \cdots \\
\end{bmatrix}
$$

- Inputs' shape is (n, 2*m)
- Layer_1's shape is (2*m, attention_size)
- Activations' shape is (n, attention_size)


### Second layer: scores <a class="anchor" id="2_1_4"></a>
This steps produces score values, one per encoder's hidden state vector.
$$
\mathbf{activations}
=
\mathbf{tanh}
\begin{bmatrix}
\cdots & \cdots    & \cdots \\
\cdots & \cdots    & \cdots \\
\cdots & activations    & \cdots \\
\vdots & \vdots    & \vdots \\
\cdots & \cdots    & \cdots \\
\end{bmatrix}
·
\begin{bmatrix}
\vdots \\
\vdots \\
Layer 2 \\
\vdots \\
\vdots \\
\end{bmatrix}
$$

- Activations' shape is (n, attention_size)
- Layer_2's shape is (attention_size, 1)

### Calculate weights: Softmax function<a class="anchor" id="2_1_5"></a>
$$
\large \alpha_{ij} = \frac{\exp{\left(e_{ij}\right)}}{\sum_{k=1}^K \exp{\left(e_{ik}\right)}}
$$

### Apply the weights to the encoder's hidden states<a class="anchor" id="2_1_6"></a>
$$
\large c_i = \sum_{j=1}^K\alpha_{ij} h_{j}
$$

### Code<a class="anchor" id="2_1_7"></a>

In [ ]:
import numpy as np

hidden_size = 16 # n
attention_size = 10
input_length = 5

encoder_states = np.random.randn(input_length, hidden_size)
decoder_state = np.random.randn(1, hidden_size)
layer_1 = np.random.randn(2*hidden_size, attention_size)
layer_2 = np.random.rand(attention_size, 1)

In [ ]:
def softmax(x, axis=0):
    """    
        axis=0 calculates softmax across rows which means each column sums to 1 
        axis=1 calculates softmax across columns which means each row sums to 1
    """
    return np.exp(x) / np.expand_dims(np.sum(np.exp(x), axis=axis), axis)

def alignment(encoder_states, decoder_state):
    inputs = np.concatenate((encoder_states, np.repeat(decoder_state, input_length, axis=0)), axis=1)
    activations = np.tanh(np.dot(inputs, layer_1))
    scores = np.dot(activations, layer_2)
    return scores

def activations(encoder_states, decoder_state):
    scores = alignment(encoder_states, decoder_state)
    weights = softmax(scores)
    weighted_scores = encoder_states * weights
    context = np.sum(weighted_scores, axis=0)
    return context

## Scaled dot product attention:<a class="anchor" id="2_2"></a>
2017's paper [Attention Is All You Need](https://arxiv.org/abs/1706.03762) introduced a different type of attention that doesn't use neural networks but only matrix product to calculate the weights to scale the inputs. However, this method uses aligned vector spaces, and that needs to be computed before the scaled dot product attention is calculated. 

It is based on three matrices, Q (Queries), K (keys) and V (Values). 
- The queries are the inputs
- The keys are the outputs
- And the values are the inputs as well. See the formula, where they are the values to weight.

The queries and keys matrices are used to calculate alignment scores that measure how well the queries and keys match. Again, a measure of similarity.

$$ 
\large \mathrm{Attention}\left(Q, K, V\right) = \mathrm{softmax}\left(\frac{QK^{\top}}{\sqrt{d_k}}\right)V
$$

- $Q·K^T$ is a measure of similarity between $Q$ and $K$. 
- Scaled down by a regularization constant $sqrt(d_k)$, with $d_k$ being the dimension of the keys.
- The softmax calculates weights
- These weights are then used to weight $V$ (usually, the inputs)

In the context of machine translation, the word embeddings, in two different languages would have been aligned separately, using other algorighms that transform the embeddings to a common space so that words with similar meanings in say en and Fre are closer together once aligned. 

One property of alignment is that they will relate words (assign higher scores for that pair) even if the words appear at different indexes in the input and the output. This should come from the fact that $Q$ and $K$ are matrices containing all input and output vectors. Therefore the dot product of $Q$ and $K$ will calculate a measure of similarity for all pair of input and output vectors.



### Code<a class="anchor" id="2_2_1"></a>

In [ ]:
def calculate_weights(queries, keys):
    dot = np.matmul(queries, keys.T)/np.sqrt(keys.shape[1])
    weights = softmax(dot, axis=1)
    
    return weights

def attention_qkv(queries, keys, values):
    return np.matmul(calculate_weights(queries, keys), values)

### Visualizing how the attention relates similar words from inputs and outputs<a class="anchor" id="2_2_2"></a>

#### Aligned vectors <a class="anchor" id="2_2_2_1"></a>
As mentioend above, scaled dot product attention assumes that aligned vectors have already been calculated by other means. Below and in the context of Neural Machine Translation (NMT) we use vocabulary dictionaries and embedding matrices. They are pre-computed and related to each other, that is, the vocabulary dictionaries map a word key to an integer index, which is the row index of this word in the embeddings matrix.

In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np

# Load dictionaries: word -> row index of word in embeddigs matrix
with open("./data/word2int_en.pkl", "rb") as f:
    en_words = pickle.load(f)
    
with open("./data/word2int_fr.pkl", "rb") as f:
    fr_words = pickle.load(f)

# Load the word embeddings: embedding at index i is related to the word dictionaries
en_embeddings = np.load("./data/embeddings_en.npz")["embeddings"]
fr_embeddings = np.load("./data/embeddings_fr.npz")["embeddings"]

#### Calculating embedding matrices for our concrete inputs and outputs <a class="anchor" id="2_2_2_2"></a>
In the context of (NMT), the inputs of the attention layer are matrices of embedding aligned vectors; one vector per word in the input and expected output. So we need to calculate these embedding matrices.

In [ ]:
def tokenize(sentence, token_mapping):
    """
    Inputs
        - sentence: str
        - token_mapping: a dictionary to map from word to int index.
    Output
        - An array of indexes for each word in the input
    """
    tokenized = []
    
    for word in sentence.lower().split(" "):
        try:
            tokenized.append(token_mapping[word])
        except KeyError:
            # Using -1 to indicate an unknown word
            tokenized.append(-1)
        
    return tokenized

def embed(tokens, embeddings):
    """
    Inputs
        - tokens: an array of indexes
        - embeddings: a matrix of shape (vocab_size, emb_dim) containing row word embeddings
    Output
        - A matrix with shape (len(tokens), emb_dim) with the word emeddings for the tokens in 'tokens'
    """
    embed_size = embeddings.shape[1]
    
    output = np.zeros((len(tokens), embed_size))
    for i, token in enumerate(tokens):
        if token == -1:
            output[i] = np.zeros((1, embed_size))
        else:
            output[i] = embeddings[token]
            
    return output

In [ ]:
# Tokenize example sentences in English and French, then get their embeddings
sentence_en = "The agreement on the European Economic Area was signed in August 1992 ."
tokenized_en = tokenize(sentence_en, en_words)
embedded_en = embed(tokenized_en, en_embeddings)

sentence_fr = "L accord sur la zone économique européenne a été signé en août 1992 ."
tokenized_fr = tokenize(sentence_fr, fr_words)
embedded_fr = embed(tokenized_fr, fr_embeddings)

# These weights indicate alignment between words in English and French
alignment = calculate_weights(embedded_fr, embedded_en)

# Visualize weights to check for alignment
fig, ax = plt.subplots(figsize=(7,7))
ax.imshow(alignment, cmap='inferno')
ax.xaxis.tick_top()
ax.set_xticks(np.arange(alignment.shape[1]))
ax.set_xticklabels(sentence_en.split(" "), rotation=90, size=16);
ax.set_yticks(np.arange(alignment.shape[0]));
ax.set_yticklabels(sentence_fr.split(" "), size=16);

Note how the correct translations are highlighted (assigned higher scores). This is because the embedding vectors in En and Fr were already aligned. But also, notice how 'signé' and 'agreement' (and the reciprocal 'accord' and 'signed') receive an also high score, again, because in the aligned space, those two embedding vectors are close together. 